<a href="https://colab.research.google.com/github/Morsalah/M.Sc-Research-HRI-using-DIGIT-tactile-sensor/blob/main/Image_Classificcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic CNN

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


Data loader test

In [ ]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from dataloader import calculate_mean_std

val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
                                        ])

# Custom dataset to load images without labels
class UnlabeledDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure 3-channel RGB
        if self.transform:
            image = self.transform(image)
        return image, img_path  # Return image + filename for reference

# Load test dataset
test_folder = "./test_data"
test_dataset = UnlabeledDataset(test_folder, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # No shuffle!


data loader

In [ ]:

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from hyperparamters_variables import Batch_size

def calculate_mean_std(loader):
    """Calculate mean and standard deviation of dataset."""
    mean = 0.
    std = 0.
    total_images_count = 0

    for images, _ in loader:
        images = images.view(images.size(0), images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += images.size(0)

    mean /= total_images_count
    std /= total_images_count

    return mean, std

def create_transformed_dataset(input_dir, transform):
    """Create dataset with transformed images."""
    transformed_dataset = ImageFolder(root=input_dir, transform=transform)
    return transformed_dataset

initial_transform = transforms.Compose([transforms.Resize(224),
                                        transforms.ToTensor()])
# Create the initial training dataset
initial_train_dataset = ImageFolder(root='./split_dataset/train',
                                    transform=initial_transform)

initial_train_loader = DataLoader(initial_train_dataset,
                                  batch_size=Batch_size,
                                  shuffle=True)

mean, std = calculate_mean_std(initial_train_loader)

# Define transformations with calculated mean and std
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

# Create transformed dataset directories for train, val, and test sets
train_dataset = create_transformed_dataset('./split_dataset/train',
                                            train_transform)

val_dataset = create_transformed_dataset('./split_dataset/val',
                                          val_test_transform)

# Create data loaders for train, val, and test datasets
train_loader = DataLoader(train_dataset,
                          batch_size=Batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=Batch_size,
                        shuffle=False)

# Verifying creation of data loaders
#print(type(train_loader), type(val_loader), type(test_loader))

'''
# Check if data loaders contain batches
for images, labels in train_loader:
    print(f"image shape {images.shape}")
    print(f"label shape {labels.shape}")
    break

print(f"Batch size of train_loader: {train_loader.batch_size}")
print(f"Batch size of val_loader: {val_loader.batch_size}")
print(f"Batch size of test_loader: {test_loader.batch_size}")

'''

heperparamters variables

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from BasicCNN import MyModel
from torchsummary import summary

# HyperParamters
Batch_size = 128
Ratio = [0.7,0.2,0.1]
Epoches = 30
Learning_rate = 0.0001

# Model Initialization
model = MyModel()

# Using Device
DEVICE = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(DEVICE)
image = (3, 224, 224)

# Loss Function and Optimizer

optimizer = Adam(model.parameters(), lr=Learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate, weight_decay=1e-4)

loss_fun = nn.BCEWithLogitsLoss()

model_architecture = summary(model,image)
# Print Model Summary
# print(summary(model,image))



loss accuracy results

In [ ]:
import os
import json
import time
from datetime import datetime
from matplotlib import pyplot as plt
from dataloader import train_loader, val_loader
from main_function import main
from hyperparamters_variables import Epoches, Learning_rate, Batch_size, Ratio, optimizer, loss_fun, model, model_architecture

def experiment(experiment_id):
    experiment_folder = f"./experiment_results/experiment_{experiment_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(experiment_folder, exist_ok=True)

    train_losses, train_accuracies, val_losses, val_accuracies = main(model, train_loader, val_loader, Epoches)

    plt.figure(figsize=(12, 5))

    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, Epoches + 1), train_losses, label="Train Loss")
    plt.plot(range(1, Epoches + 1), val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss vs Epochs")
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, Epoches + 1), train_accuracies, label="Train Accuracy")
    plt.plot(range(1, Epoches + 1), val_accuracies, label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Epochs")
    plt.legend()

    # Save the plot before displaying
    plot_filename = os.path.join(experiment_folder, "experiment_plots.png")
    plt.savefig(plot_filename)  # Save the plot

    plt.close()  # Close the figure to free memory

    # Prepare experiment details
    experiment_details = {
        "Experiment_ID": experiment_id,
        "Train_Test_Split": f"train={Ratio[0]}, val={Ratio[1]}, test={Ratio[2]}",
        "Model": print(model_architecture),
        "Optimizer": optimizer.__class__.__name__,
        "Loss_Function": loss_fun.__class__.__name__,
        "Learning_Rate": Learning_rate,
        "Epochs": Epoches,
        "Batch_Size": Batch_size,
        "Train_Losses": train_losses,
        "Train_Accuracies": train_accuracies,
        "Val_Losses": val_losses,
        "Val_Accuracies": val_accuracies
    }

    # Save experiment details as a JSON file
    json_filename = os.path.join(experiment_folder, "experiment_details.json")
    with open(json_filename, 'w') as f:
        json.dump(experiment_details, f, indent=4)

    print(f"Experiment results saved in {experiment_folder}")
    return experiment_folder

if __name__ == "__main__":
    experiment_id = f"exp_{int(time.time())}"  # Generates a unique ID based on the current timestamp
    experiment_folder = experiment(experiment_id)


main function

In [ ]:
import torch
from train_function import train
from validation_function import validation
from hyperparamters_variables import Epoches

def main(model, train_loader, val_loader,Epoches):

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(Epoches):
        train_loss, train_accuracy = train(model, train_loader)
        val_loss, val_accuracy = validation(model, val_loader)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1:0>2}/{Epoches} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% - Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    return train_losses, train_accuracies, val_losses, val_accuracies

first CNN implementation

In [ ]:
'''
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            # block2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            # block3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            # block4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )

        self.flatten = nn.Flatten()

        # Compute the in_features dynamically
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 224, 224)
            dummy_output = self.conv_layers(dummy_input)
            self.in_features = dummy_output.view(1, -1).shape[1]

        self.fc_layers = nn.Sequential(
            nn.Linear(self.in_features, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x
'''

'''
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

'''

performance results

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from hyperparamters_variables import DEVICE

def predict_images(model, test_loader):
    model.eval()
    model.to(DEVICE)

    predicted_labels, predicted_probs, image_paths = [], [], []

    for images, paths in test_loader:
        images = images.to(DEVICE)

        with torch.inference_mode():
            outputs = model(images)
        prob = torch.sigmoid(outputs)
        pred_indices = (prob > 0.5).long().squeeze()
        pred_probs = prob.squeeze()

        # Store results
        predicted_labels.extend(pred_indices.cpu().numpy())
        predicted_probs.extend(pred_probs.cpu().numpy())
        image_paths.extend(paths)

    return image_paths, np.array(predicted_labels), np.array(predicted_probs)


train function

In [ ]:

import torch
import torch.nn as nn
from hyperparamters_variables import optimizer,loss_fun,DEVICE,model
from tqdm import tqdm

def train(model, train_loader):
    model.train()
    model.to(DEVICE)
    running_loss = 0
    correct_predictions = 0
    total_train_samples = 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fun(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Convert logits to probabilities and apply threshold
        predictions = torch.sigmoid(outputs) > 0.5
        correct_predictions += (predictions == labels.unsqueeze(1)).sum().item()
        total_train_samples += labels.size(0)

    train_avg_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_predictions / total_train_samples

    return train_avg_loss, train_accuracy


validation function

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from hyperparamters_variables import loss_fun,DEVICE,model

def validation(model, val_loader):
    model.eval()
    model.to(DEVICE)

    running_loss = 0
    correct_predictions = 0
    total_val_samples = 0

    for images, labels in tqdm(val_loader, desc="Validation"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        with torch.no_grad():
             outputs = model(images)

        loss = loss_fun(outputs, labels.float().unsqueeze(1))
        running_loss += loss.item()
        #_, predicted = torch.max(outputs.data, dim=1)
        predicted = torch.round(torch.sigmoid(outputs.data)).squeeze(1)
        #labels = labels.squeeze(1)
        total_val_samples += labels.shape[0]
        correct_predictions += (predicted == labels).sum().item()

    val_avg_loss = running_loss / len(val_loader)
    val_accuracy = 100 * correct_predictions / total_val_samples
    return val_avg_loss, val_accuracy